## LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import re
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# driver = webdriver.Chrome()
# base_url = 'https://mubi.com/en/films?filterShowing=false&sort=popularity_quality_score&page='

## TITLES SCRAPING

In [13]:
# import time
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# driver = webdriver.Chrome()

# base_url = 'https://mubi.com/en/films?filterShowing=false&sort=popularity_quality_score&page='

# # List to store all titles
# all_titles = []

# # Scrape pages 1 to 3 (inclusive)
# for page in range(1, 417):
#     url = f"{base_url}{page}"
#     driver.get(url)

#     # Wait for the titles to load
#     wait = WebDriverWait(driver, 5)
#     title_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'h3.css-1auskk6.e1rguf1r1')))
    
#     # Extract titles and add them to the list
#     for ele in title_elements:
#         all_titles.append(ele.text)

#     # Wait a few seconds before moving to the next page
#     time.sleep(5)

# # Close the browser
# driver.quit()

# # Print all titles
# print(all_titles)

['CLOSE', 'PERFECT DAYS', 'AFTERSUN', 'IN THE MOOD FOR LOVE', 'MY LIFE AS A COURGETTE', 'PORTRAIT OF A LADY ON FIRE', "WHERE IS THE FRIEND'S HOUSE?", "LEILA'S BROTHERS", 'THE GODFATHER', 'THE GLEANERS AND I', 'DRIVE MY CAR', 'THE 400 BLOWS', 'CHUNGKING EXPRESS', '2001: A SPACE ODYSSEY', 'PARIS, TEXAS', 'LA HAINE', 'NIGHT AND FOG', 'SEVEN SAMURAI', 'APOCALYPSE NOW', 'STALKER', 'LA JETÉE', 'SO LONG, MY SON', 'PERSONA', 'ALL ABOUT MY MOTHER', 'PULP FICTION', 'THE BEACHES OF AGNÈS', 'THREE COLORS: RED', 'THE GODFATHER: PART II', 'SPIRITED AWAY', 'BICYCLE THIEVES', 'MODERN TIMES', 'FANTASTIC MR. FOX', 'FESTEN', 'MEETING THE MAN: JAMES BALDWIN IN PARIS', '8½', 'BLUE', 'THE WORST PERSON IN THE WORLD', 'TAXI DRIVER', 'THREE COLORS: BLUE', 'LOVE, DEUTSCHMARKS AND DEATH', 'DR. STRANGELOVE', 'RAN', 'A CLOCKWORK ORANGE', 'CITIZEN KANE', 'LAURENCE ANYWAYS', 'A TRIP TO THE MOON', 'TRIPPING WITH NILS FRAHM', 'COME AND SEE', 'PARASITE', 'REAR WINDOW', 'THE SHINING', 'PSYCHO', 'INCENDIES', 'WINGS OF DE

In [22]:
# titles = pd.DataFrame(all_titles).reset_index(drop='first')

In [22]:
# titles.to_csv('titles.csv')

In [2]:
import pandas as pd
titles_m = pd.read_csv('titles.csv')
len(titles_m)

9982

## PLAYLIST SCRAPING

In [112]:
# import time
# import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# Initialize the Chrome driver
driver = webdriver.Chrome()

# URL containing the playlists
pl_base_url = "https://mubi.com/en/lists?page="

# Lists to store data
playlists_titles = []
playlist_followers = []
playlist_links = []

for page in range(1, 32844):
    pl_url = f"{pl_base_url}{page}"
    driver.get(pl_url)

    # Find all playlist links
    wait = WebDriverWait(driver, 5)
    
    playlist_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h2.css-1gj19i2.eq7xvik9")))
    for ele in playlist_elements:
        playlists_titles.append(ele.text)
    
    ele_followers = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"span.css-f7pjix.eq7xvik14")))
    for ele in ele_followers:
        playlist_followers.append(ele.text)
        
    ele_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.css-15c7jmc.eq7xvik3')))
    for ele in ele_links:
        link = ele.get_attribute("href")
        playlist_links.append(link)

    time.sleep(6)  # Robot 5 sec min

# Close the browser
driver.quit()


TimeoutException: Message: 


In [44]:
pd.DataFrame(playlists_titles).reset_index(drop='first').to_csv('playlist_titles.csv')
pd.DataFrame(playlist_followers).reset_index(drop='first').to_csv('playlist_followers.csv')
pd.DataFrame(playlist_links).reset_index(drop='first').to_csv('playlist_links_tot.csv')

## CREATING DISTINCT VALUES FOR TITLES

1) Import Database (sqlite to sql to) "movies.sql"

2) Standardize data formatting for movies' titles

3) Merge with "titles.csv" for unique values

In [13]:
# Create a dataframe from the scraped titles with only the titles
# Apply title formatting to the values in column
import string
df = pd.read_csv('titles.csv')
df.columns = ['index','titles']
df['titles'] = df['titles'].apply(lambda x: string.capwords(x)).copy()
df['titles']
df = df.drop(df.columns[0], axis=1)

In [14]:
# Save the dataframe
df.to_csv('titles.csv')

In [15]:
# sqlite database https://www.kaggle.com/datasets/clementmsika/mubi-sqlite-database-for-movie-lovers
movies = pd.read_csv('C:/Users/benja/Downloads/mubi_db.sqlite/movies.csv')  

In [16]:
# import apply identical formatting
movies['movie_title']= movies['movie_title'].apply(lambda x: string.capwords(x)).copy()

In [17]:
# save the dataframe
movies.to_csv('movies_test.csv')

In [18]:
# len(df['titles']) is about 10k
# len(movies['movie_title']) is about 220k

216593

In [19]:
combine_movies = pd.concat([movies['movie_title'], df['titles']], ignore_index=True, sort=False)
combine_movies = combine_movies.drop_duplicates(keep='first')

# DECISION MADE: at this point the model won't recognize if a movie has two versions 
# REASON: new titles scraped don't have a movie_ID

In [20]:
print(len(combine_movies))
combine_movies.reset_index(drop='first').to_csv('combine_movies.csv')

196762


In [17]:
combine_movies = pd.read_csv('combine_movies.csv').drop('Unnamed: 0', axis=1)
# combine_movies = pd.DataFrame(combine_movies)
combine_movies.columns = ['title']
combine_movies

,title
0,La Antena
1,Elementary Particles
2,It's Winter
3,Kirikou And The Wild Beasts
4,Padre Nuestro
...,...
196757,The Sparks Brothers
196758,"Orlando, My Political Biography"
196759,Hitler's Hollywood: German Cinema In The Age O...
196760,I And The Stupid Boy


## MOVIE TITLES IN PLAYLIST

For time and computation capacity I used a test version with only the first 48 movies displayed for each page. Hence below you find the code to scrape the full url 

In [ ]:
from selenium.common.exceptions import NoSuchElementException, WebDriverException

driver = webdriver.Chrome()
database_of_playlists = {}

# Loop through each playlist link
for link in playlist_links_list_3k:
    try:
        driver.get(link)  # Visit the playlist URL
        
        wait = WebDriverWait(driver, 12)
        
        # Loop to click the "Show more" button until it's no longer present
        while True:
            try:
                show_more_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.e19wocv63.e150chai0.css-7xwb6j.e46vom80')))
                show_more_button.click()
                time.sleep(5)  # Wait for the page to load more elements
            except NoSuchElementException:
                break  # No more "Show more" button to click
        
        # Scrape the elements after all have been loaded
        title_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'h3.css-1auskk6.e1rguf1r1')))
        database_of_playlists[link] = [title_element.text for title_element in title_elements]
    
    except WebDriverException as e:
        if "disconnected: not connected to DevTools" in str(e):
            print(f"Driver disconnected while processing link {link}. Attempting to reconnect...")
            driver.quit()
            driver = webdriver.Chrome()  # Reinitialize the driver
            continue
        else:
            print(f"Error occurred while processing link {link}: {e}")
            continue
    finally:
        time.sleep(6)
        
# Close the browser
driver.quit()

# Convert the dictionary into a DataFrame
final_dataframe = pd.DataFrame.from_dict(database_of_playlists, orient='index')


## CHECK IF MOVIE TITLES ARE IN PLAYLIST LINKS

Check if the titles in combine_movies are present in each playlist. As it was a test model and there was an urgent need for the output I chose to proceed with a VBA Macro and act directly in the dataframe.